# Mask Tutorial

In this tutorial, we go over the importance of masks, as well as how they are made and used. 

In [ ]:
import specpolFlow as pol
import matplotlib.pyplot as plt
import numpy as np

# What is a Mask?

Analytically, a mask is a function with dirac deltas at wavelengths corresponding to specific spectral line. The amplitude of the dirac delta function corresponds to the line depth. Numerically, a mask is an array of zeros with a depth at the center of each line. Thus, the mask tells us the location and depth of all lines in a spectrum but does not tell us about the shape of the lines of the spectrum as a whole.

# Why do we care?

Given an LSD profile and a mask, we can convolve the LSD profile with the mask to get the spectrum. Typically, though, we have the spectrum and a mask but want the LSD profile. This process of going from a spectrum and a line mask to an LSD profile can be done by doing the reverse operation called a deconvolution. We need a mask to help us weigh each spectral line in the spectrum so that they can be averaged together in an LSD.

Note that the hydrogen lines are automatically exluded when `atomsOnly` is `True`. This is done since the hydrogen lines are a different shape than all the other lines in the spectrum due to their broad wings.

# Mask Creation

To create a mask we will use the `make_mask` function. The function has the following inputs:
* lineListFile - name of the file containing the line list
* maskFile - name of the file to write the output mask to
* depthCutoff - Only include lines in the mask that are deeper than this value
* wlStart - Optional, use only lines with wavelengths above this
* wlEnd - Optional, use only lines with wavelengths below this
* landeStart - Optional, use only lines with effective Lande factors above this
* landeEnd - Optional, use only lines with effective Lande factors below this
* elementsUsed - Optional, list of elements to include in the mask creation
* elementsExclude - Optional, list of elements to exlude from the mask creation
* atomsOnly - Only include atomic lines (no molecular lines and no H-lines)
* includeNoLande - Include lines in the mask even if they don't have an effective Lande factor given
* DefaultLande - The value for the effective Lande factor to use in the case that a line has no listed effective Lande factor 

For most cases you will only use the top three inputs and `atomsOnly`. The input line list is a VALD line list file and is obtained from the VALD website. More details about VALD is given in `SpecPolTutorial.ipynb`. The `make_mask` function will automatically attempt to calculate the effective Lande factor for lines where it is not given, however if one is unable to be calculated it will be excluded if `includeNoLande` is `False` or it will equal `DefaultLande` if `includeNoLande` is `True`.

In the example case used below, we are using all atomic lines in the line list except ones without effective Lande factors and the H-lines. 


In [ ]:

file_name = 'specpolFlow_tutorialfiles/LongList_T27000G35.dat'
file_output = 'specpolFlow_tutorialfiles/test_output/T27000G35_depth0.02.mask'
mask_file = pol.makeMask.make_mask(file_name, file_output, depthCutoff = 0.02, atomsOnly = True)

# Mask Cleaning

Once we have our mask, we need to clean it. Mask cleaning is the removal of lines that we don't want to be used in the LSD profile. Typically, excluded lines include all lines within the Telluric regions as well as lines within the H wings. Lines within the Telluric regions are contaminated by Earth's atmosphere and thus are not useable. Lines in the H wings blended with Hydrogen lines thus they are contaminated and unusable since they have a different shape (will effect averaging). Additionally, when dealing with stars with emission, care should be taken to exclude the emission lines if they have different shapes.

To clean our masks we first want to define our `excluded_regions`. These are the regions within which no lines will be used. This will be structured as a dictionary with two keys, `WLStart` and `WLFinish`. Each of these keys is a list of wavelengths to mark the begining and end of the exclusion regions. This dictionary can either be created manually or the default regions can be used by specifying a velcity range around the H-lines to be excluded and then using the `default_exclude_regions` function as done below. 

In [ ]:
# inputs
velrange = 600 # units are in km/s
excluded_regions = pol.cleanMask.default_exclude_regions(velrange)
excluded_regions

Once we have our excluded regions, we can clean the mask using the `clean_model_mask` function. This function takes in the name of the uncleaned mask, the name of the output mask file, and the dictionary containing the excluded regions. The ouput is a cleaned line mask; lines lying within the `excluded_regions` have been removed.

In [ ]:
# Location of the output file
clean_mask = 'specpolFlow_tutorialfiles/test_output/hd46328_test_depth0.02_clean.mask'

# Cleaning the mask
pol.cleanMask.clean_model_mask(file_output, clean_mask, excluded_regions)

The `cleanMask` module also includes a visualization of the contaminated regions called `Excluded_Regions_Visual`. This function takes in the spectrum file, the clean mask and the excluded regions dictionary, and outpus a plot showing the excluded regions in grey and the exluded lines in red, while keeping included lines in green.

In [ ]:
# for Spectrum and excluded regions visualization
spectrum = 'specpolFlow_tutorialfiles/hd46328_test_1.s'
pol.cleanMask.Excluded_Regions_Visual(spectrum, clean_mask, excluded_regions)   